In [14]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
le = LabelEncoder()
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
import tensorflow as tf
from tensorflow import keras

In [2]:
data = pd.read_csv('/kaggle/input/loan-approval/loanapproval.csv')
data.head()

,age,marital,Personal_loan,housing_loan,ever_defaulted,Loan_approved
0,56,married,no,no,no,no
1,57,married,no,no,unknown,no
2,37,married,yes,no,no,no
3,40,married,no,no,no,no
4,56,married,no,yes,no,no


In [3]:
# Checking for Missing values 
data.isnull().sum()

age               0
marital           0
Personal_loan     0
housing_loan      0
ever_defaulted    0
Loan_approved     0
dtype: int64

In [4]:
# convert string variable to One Hot Encoding
dummyfied_data = data.apply(le.fit_transform)
dummyfied_data.head()

,age,marital,Personal_loan,housing_loan,ever_defaulted,Loan_approved
0,39,1,0,0,0,0
1,40,1,0,0,1,0
2,20,1,2,0,0,0
3,23,1,0,0,0,0
4,39,1,0,2,0,0


In [5]:
# Scalling the numeric column
col_to_scale =['age']
dummyfied_data[col_to_scale] = scaler.fit_transform(dummyfied_data[col_to_scale])
dummyfied_data.head()

,age,marital,Personal_loan,housing_loan,ever_defaulted,Loan_approved
0,0.506494,1,0,0,0,0
1,0.519481,1,0,0,1,0
2,0.259740,1,2,0,0,0
3,0.298701,1,0,0,0,0
4,0.506494,1,0,2,0,0


In [7]:
X = dummyfied_data.drop('Loan_approved',axis = 1)
Y = dummyfied_data[['Loan_approved']]

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size = 0.20, 
random_state = 42)

In [9]:
test_df = pd.concat ([X_test,y_test],axis =1 )

In [10]:
# Adaboost classifier
abc = AdaBoostClassifier(random_state=0)
abc_model = abc.fit(X_train, y_train)
abc_pred = abc_model.predict(X_test)
test_df['abc_pred'] = abc_pred
# test_df.to_csv('classification_results.csv')
score = accuracy_score(y_test, abc_pred)
score

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.8858946346200534

In [11]:
# creating a RF classifier, training a random Forest model
clf = RandomForestClassifier(random_state = 42) 
clf.fit(X_train, y_train)
# Predicting on Test data and checking the accuracy. This can be extended to train data as well
rf_pred = clf.predict(X_test)
test_df['rf_pred'] = rf_pred
score = accuracy_score(y_test, rf_pred)
score

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


0.883709638261714

In [12]:
# Gradient boosting classifier
gbcl = GradientBoostingClassifier(random_state = 42)
gbcl.fit(X_train, y_train)
gbcl_pred = gbcl.predict(X_test)
test_df['gbcl_pred'] = gbcl_pred
# test_df.to_csv('classification_results.csv')
score = accuracy_score(y_test, gbcl_pred)
score

/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.8852876911871813

In [13]:
# XGB
xgb = XGBClassifier(random_state = 42)
xgb.fit(X_train, y_train)
xgb_pred = xgb.predict(X_test)
test_df['xgb_pred'] = xgb_pred
score = accuracy_score(y_test, xgb_pred)
score

0.8852876911871813

In [17]:
#ANN
model = keras.Sequential([
    keras.layers.Dense(3, input_shape = (5,), activation = 'relu'),
    keras.layers.Dense(2, input_shape = (3,), activation = 'relu'),
    keras.layers.Dense(1, activation = 'sigmoid'),
])
model.compile(optimizer = 'adam',
              loss = 'binary_crossentropy',
              metrics = ['accuracy']
)
model.fit(X_train, y_train, epochs = 5)
model.evaluate(X_test, y_test,)
yp = model.predict(X_test)
y_pred = []
for element in yp:
    if element>0.5:
        y_pred.append(1)
    else:
        y_pred.append(0)
test_df['y_pred'] = y_pred
score = accuracy_score(y_test, y_pred)
score

Epoch 1/5
1030/1030 [==============================] - 2s 2ms/step - loss: 0.4299 - accuracy: 0.8420
Epoch 2/5
1030/1030 [==============================] - 2s 2ms/step - loss: 0.3547 - accuracy: 0.8876
Epoch 3/5
1030/1030 [==============================] - 2s 2ms/step - loss: 0.3502 - accuracy: 0.8876
Epoch 4/5
1030/1030 [==============================] - 2s 2ms/step - loss: 0.3479 - accuracy: 0.8876
Epoch 5/5
258/258 [==============================] - 1s 1ms/step - loss: 0.3478 - accuracy: 0.8865


0.8865015780529255

In [15]:
# Ensemble the ensembles
column_names = ['abc_pred', 'rf_pred', 'gbcl_pred']
test_df['sum'] = test_df[column_names].sum(axis=1)
test_df['ensembeled_pred'] = [1 if x > 1 else 0 for x in test_df['sum'] ]
score = accuracy_score(y_test, test_df['ensembeled_pred'])
score

0.8856518572469045